<a href="https://colab.research.google.com/github/cis9650group7-boop/Group-7_Project1_HealthRateAnalysis/blob/dev/term_project_group7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Analysis of Health Rate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Author: CIS 9650 Group 7 (Anish Bijusoman, Ivana Sundararao, Qingrong Tan, Reem Hussein)
### Date : November 28th, 2025

## Executive Summary

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

## Introduction

## Problem Statement / Research Question

## Data Description

## Setup and Environment

In [ ]:
!pip install google-cloud-storage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
from google.cloud import storage
from io import BytesIO

## Data Loading

### Functions

In [ ]:
from google.cloud import storage
import pandas as pd
from io import BytesIO
import os
import json
import requests
import re # Import regex module for parsing HTML

def list_public_bucket(bucket_name, course, project):
    prefix = f"{course}/Project {project}/"

    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blobs = client.list_blobs(bucket, prefix=prefix)
    print(f"Listing files in public bucket '{bucket_name}' under '{prefix}':")
    filenames = []
    for blob in blobs:
        print(blob.name)
        filenames.append(blob.name)

    return filenames

def gdrive_file_to_dataframe(file_id: str, file_type: str):
    """
    Downloads a file from Google Drive using its ID and returns a pandas DataFrame.
    Handles Google Drive virus scan warnings by parsing the HTML form to get the
    actual download URL.
    """
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()
    response = session.get(URL, params={'id': file_id}, stream=True)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

    # Check if the response content is the virus scan warning HTML page
    if "Google Drive - Virus scan warning" in response.text:
        print("Google Drive virus scan warning detected. Attempting to bypass...")
        # Extract the download action URL and hidden input parameters from the HTML form
        match = re.search(r'<form id="download-form" action="([^"]+)" method="get">', response.text)
        if not match:
            raise ValueError("Could not find download form in virus warning page.")
        download_action_url = match.group(1)

        hidden_inputs = re.findall(r'<input type="hidden" name="([^"]+)" value="([^"]*)">', response.text)
        download_params = {name: value for name, value in hidden_inputs}

        # Make the actual download request with the extracted parameters
        final_response = session.get(download_action_url, params=download_params, stream=True)
        final_response.raise_for_status()
        data = BytesIO(final_response.content)
    else:
        # No virus warning, proceed with the initial response content
        data = BytesIO(response.content)

    df = None
    if file_type == "csv":
        df = pd.read_csv(data, engine='python', on_bad_lines='warn')
    elif file_type in ["xls", "xlsx"]:
        df = pd.read_excel(data)
    elif file_type == "parquet":
        df = pd.read_parquet(data)
    elif file_type == "json":
        text = data.read().decode("utf-8")
        json_obj = json.loads(text)
        if isinstance(json_obj, list):
            df = pd.DataFrame(json_obj)
        elif isinstance(json_obj, dict):
            df = pd.json_normalize(json_obj)
        else:
            raise ValueError("Unsupported JSON structure")
    else:
        raise ValueError(f"Unsupported file type: {file_type}")

    return df

# Fetching the data from the provided Google Drive link
try:
    file_id_to_fetch = '1yJjVih68D_J4JON2LXMLGIh_FoeibwaG'
    # Assuming the file type is CSV based on the filename provided
    gdrive_df = gdrive_file_to_dataframe(file_id_to_fetch, 'csv')
    print("Successfully fetched and loaded data from Google Drive into 'gdrive_df'. Head of the DataFrame:")
    print(gdrive_df.head())
except Exception as e:
    print(f"Error fetching or loading data from Google Drive: {e}")


Google Drive virus scan warning detected. Attempting to bypass...
Successfully fetched and loaded data from Google Drive into 'gdrive_df'. Head of the DataFrame:
                        _id                                           slug  \
0  647e89e399dca372f92a0d7c    3183031/Dr-Janan+S.-Sayyed-NEW+YORK-NY.html   
1  647e89e499dca372f92a0d91      3345163/Dr-SHIRIN-PETERS-NEW+YORK-NY.html   
2  647e89e499dca372f92a0db9  3081147/Dr-THOMAS+J.-MULHERN-New+York-NY.html   
3  647e89e499dca372f92a0dba      3081144/Dr-ZINA-RUTKIN-GREAT+NECK-NY.html   
4  647e89e499dca372f92a0dbd  3081128/Dr-JOHN+S.-CAVALLARO-Brooklyn-NY.html   

                            created                          modified      id  \
0  2014-06-13T11:39:19.399618-04:00  2023-04-16T10:17:11.091386-04:00  870323   
1  2014-06-13T10:53:51.310523-04:00  2022-11-15T15:19:02.430196-05:00  819737   
2  2014-06-13T12:33:17.025883-04:00  2022-03-08T14:16:12.357691-05:00  939574   
3  2014-06-13T12:33:17.131851-04:00  2022-03-

## Data Preparation

In [ ]:
df = gdrive_df

print("Number of rows", len())
print("Number of columns", len(df.columns))

Number of rows 143791
Number of columns 1611


In [ ]:
# datatypes available
print("The datatypes available are:")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df.dtypes)

The datatypes available are:
_id                                                               object
slug                                                              object
created                                                           object
modified                                                          object
id                                                                 int64
facet_url                                                         object
full_name                                                         object
full_name_specialty                                               object
location.id                                                        int64
location.category                                                 object
location.created                                                  object
location.modified                                                 object
location.longitude                                               float64
location.latitude     

In [ ]:
# similar naming columns
pd.set_option('display.max_columns', None)
df.columns = df.columns.str.lower().str.replace('_', '')

list_cols= list(df.columns)
print("The columns are:")
print(list_cols)

The columns are:
['id', 'slug', 'created', 'modified', 'id', 'faceturl', 'fullname', 'fullnamespecialty', 'location.id', 'location.category', 'location.created', 'location.modified', 'location.longitude', 'location.latitude', 'location.slug', 'location.city.id', 'location.city.provincename', 'location.city.provinceslug', 'location.city.countryname', 'location.city.countryslug', 'location.city.coverimages.253x83', 'location.city.coverimages.autoxauto', 'location.city.modified', 'location.city.created', 'location.city.deleted', 'location.city.slug', 'location.city.name', 'location.city.coverimage', 'location.city.province', 'location.name', 'location.address', 'location.suite', 'location.postalcode', 'location.cityid', 'location.phonenumber', 'location.website', 'location.image', 'location.images.autoxauto', 'location.images.77x77', 'location.images.100x100', 'location.images.165x165', 'location.images.70x70', 'location.rating.cleanliness', 'location.rating.facilities', 'location.rating.

1. These need not be changed:'id', 'slug', 'created', 'modified', 'id', 'faceturl', 'fullname', 'fullnamespecialty'

2. Need to change:
-

## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

1. Barber, David. Bayesian Reasoning and Machine Learning. Cambridge University Press, 2012.
2. Aste, Tomaso, Paola Cerchiello, and Roberta Scaramozzino. "Information-Theoretic Causality Detection between Financial and Sentiment Data."Entropy, vol. 24, no. 6, 2022, pp. 1–18. DOI:10.3390/e24060774.
3. Metz, Cade. "Microsoft Puts OpenAI’s Sam Altman in Charge of New Advanced AI Research Team."
The New York Times, 20 Nov. 2023, www.nytimes.com/2023/11/20/technology/openai-microsoft-altman.html

## Appendix